# Day10_0: A/B 테스트 이론 (A/B Test Theory)

## 📚 학습 목표

**Part 1: 실험 설계**
1. A/B 테스트의 기본 개념과 목적 이해하기
2. 대조군(Control)과 실험군(Treatment) 구분하기
3. 무작위 배정(Randomization)의 중요성 파악하기
4. 표본 크기(Sample Size) 계산 방법 배우기
5. 실험 기간 설정 기준 이해하기

**Part 2: 평가 지표와 통계적 유의성**
1. 전환율(Conversion Rate), 클릭률(CTR), 평균 주문 금액(AOV) 계산하기
2. p-value와 통계적 유의성 해석하기
3. 검정력(Statistical Power)과 효과 크기(Effect Size) 이해하기
4. A/B 테스트 실무 함정(Pitfalls) 피하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| A/B 테스트 | 데이터 기반 의사결정 | "버튼 색상 변경이 구매를 늘릴까?" |
| 표본 크기 계산 | 실험 신뢰도 확보 | "몇 명의 사용자가 필요한가?" |
| 전환율 분석 | 마케팅 효과 측정 | 랜딩페이지 A vs B 성과 비교 |
| 통계적 유의성 | 우연 vs 실제 효과 구분 | "이 차이가 진짜인가?" |
| 실험 설계 | 편향 없는 결과 도출 | 공정한 테스트 환경 구축 |

**분석가 관점**: A/B 테스트는 "감"이 아닌 "데이터"로 의사결정하는 가장 강력한 도구입니다!

---

# Part 1: 실험 설계

---

## 1.1 A/B 테스트란?

### 정의

**A/B 테스트**(또는 분할 테스트, Split Test)는 두 가지 버전(A와 B)을 비교하여 어떤 것이 더 효과적인지 **통계적으로 검증**하는 실험 방법입니다.

### A/B 테스트 프로세스

```
1. 가설 설정 → 2. 실험 설계 → 3. 표본 크기 계산 → 4. 무작위 배정 → 5. 실험 실행 → 6. 결과 분석 → 7. 의사결정
```

In [3]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest, proportion_effectsize
from statsmodels.stats.power import NormalIndPower, TTestIndPower
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


### 💡 실무 예시: 이커머스 결제 버튼 A/B 테스트

**상황**: 온라인 쇼핑몰에서 결제 버튼 색상을 변경하면 전환율이 올라갈까?

- **가설 (H₀)**: 기존 파란 버튼과 신규 초록 버튼의 전환율에 차이가 없다
- **대립가설 (H₁)**: 신규 초록 버튼의 전환율이 더 높다

In [ ]:
# A/B 테스트 시나리오 예시
ab_test_scenario = {
    '항목': ['테스트 대상', '대조군 (A)', '실험군 (B)', '성공 지표', '실험 기간', '표본 크기'],
    '내용': [
        '결제 버튼 색상',
        '기존 파란 버튼',
        '신규 초록 버튼',
        '전환율 (구매 완료 / 버튼 클릭)',
        '2주',
        '각 그룹 5,000명'
    ]
}

scenario_df = pd.DataFrame(ab_test_scenario)
print("A/B 테스트 설계서")
print("=" * 50)
print(scenario_df.to_string(index=False))

---

## 1.2 대조군(Control)과 실험군(Treatment)

### 용어 정리

| 용어 | 다른 표현 | 설명 |
|------|----------|------|
| 대조군 (Control) | A그룹, Baseline | 기존 버전, 변화 없음 |
| 실험군 (Treatment) | B그룹, Variant | 새로운 버전, 변화 적용 |

### 핵심 원칙

1. **단일 변수 변경**: 한 번에 하나의 요소만 변경 (버튼 색상만 다르게, 문구는 동일)
2. **동시 진행**: A와 B를 같은 기간에 테스트 (시간에 따른 외부 영향 제거)
3. **동등한 조건**: 트래픽을 50:50으로 분배

In [ ]:
# 대조군 vs 실험군 데이터 시뮬레이션
np.random.seed(42)

# 시나리오: 웹사이트 전환율 테스트
n_control = 1000  # 대조군 방문자
n_treatment = 1000  # 실험군 방문자

# 전환율 설정 (대조군 3%, 실험군 4%)
control_rate = 0.03
treatment_rate = 0.04

# 전환 시뮬레이션 (베르누이 분포)
control_conversions = np.random.binomial(1, control_rate, n_control)
treatment_conversions = np.random.binomial(1, treatment_rate, n_treatment)

print("A/B 테스트 데이터 요약")
print("=" * 50)
print(f"대조군 (A): {n_control}명 방문, {control_conversions.sum()}명 전환")
print(f"          전환율: {control_conversions.mean():.2%}")
print(f"실험군 (B): {n_treatment}명 방문, {treatment_conversions.sum()}명 전환")
print(f"          전환율: {treatment_conversions.mean():.2%}")
print(f"\n전환율 차이: {treatment_conversions.mean() - control_conversions.mean():.2%}p")

### 💡 실무 예시: 앱 UI 변경 테스트

In [ ]:
# 앱 UI 변경 A/B 테스트 결과
ui_test_data = {
    '그룹': ['대조군 (기존 UI)', '실험군 (신규 UI)'],
    '사용자 수': [5000, 5000],
    '회원가입 수': [150, 200],
    '전환율': [3.0, 4.0]
}

ui_df = pd.DataFrame(ui_test_data)
ui_df['전환율(%)'] = ui_df['전환율'].apply(lambda x: f"{x:.1f}%")

print("앱 UI A/B 테스트 결과")
print(ui_df[['그룹', '사용자 수', '회원가입 수', '전환율(%)']].to_string(index=False))

# 상대적 개선율 (Relative Lift)
relative_lift = (4.0 - 3.0) / 3.0 * 100
print(f"\n상대적 개선율: {relative_lift:.1f}% 증가")

---

## 1.3 무작위 배정 (Randomization)

### 왜 무작위 배정이 중요한가?

**선택 편향(Selection Bias)** 방지:
- 특정 유형의 사용자가 한쪽 그룹에 몰리면 결과가 왜곡됨
- 예: VIP 고객만 신규 UI를 보면, 전환율 상승이 UI 때문인지 VIP 때문인지 알 수 없음

### 무작위 배정 방법

1. **단순 무작위 배정**: 각 사용자를 50% 확률로 A/B 배정
2. **층화 무작위 배정**: 특정 속성(연령, 지역 등) 기준으로 균형 맞춤
3. **해시 기반 배정**: 사용자 ID를 해시하여 일관된 그룹 배정

In [ ]:
# 무작위 배정 시뮬레이션
import hashlib

def assign_group_random(user_id, ratio=0.5):
    """단순 무작위 배정"""
    return 'B' if np.random.random() < ratio else 'A'

def assign_group_hash(user_id, ratio=0.5):
    """해시 기반 일관된 배정 (같은 사용자는 항상 같은 그룹)"""
    hash_value = int(hashlib.md5(str(user_id).encode()).hexdigest(), 16)
    return 'B' if (hash_value % 100) < (ratio * 100) else 'A'

# 1000명 사용자 배정 테스트
user_ids = range(1, 1001)

# 해시 기반 배정 (일관성 확인)
assignments = [assign_group_hash(uid) for uid in user_ids]
group_a = assignments.count('A')
group_b = assignments.count('B')

print("무작위 배정 결과 (해시 기반)")
print("=" * 40)
print(f"A 그룹: {group_a}명 ({group_a/10:.1f}%)")
print(f"B 그룹: {group_b}명 ({group_b/10:.1f}%)")

# 동일 사용자 재배정 확인
print(f"\n사용자 123의 그룹: {assign_group_hash(123)}")
print(f"사용자 123의 그룹 (재확인): {assign_group_hash(123)}")
print("(동일한 결과가 나옴 - 일관성 보장)")

### 💡 실무 예시: 층화 무작위 배정

In [ ]:
# 층화 무작위 배정: 연령대별로 균형 맞추기
np.random.seed(42)

# 사용자 데이터 생성
users = pd.DataFrame({
    'user_id': range(1, 1001),
    'age_group': np.random.choice(['20대', '30대', '40대', '50대+'], 1000, p=[0.3, 0.35, 0.25, 0.1])
})

# 연령대별 층화 배정
def stratified_assign(df):
    """각 연령대 내에서 50:50 배정"""
    result = []
    for age_group in df['age_group'].unique():
        group_df = df[df['age_group'] == age_group].copy()
        n = len(group_df)
        assignments = ['A'] * (n // 2) + ['B'] * (n - n // 2)
        np.random.shuffle(assignments)
        group_df['group'] = assignments
        result.append(group_df)
    return pd.concat(result)

users = stratified_assign(users)

# 층화 배정 결과 확인
balance_check = users.groupby(['age_group', 'group']).size().unstack(fill_value=0)
print("층화 무작위 배정 결과 (연령대별 A/B 그룹 분포)")
print("=" * 50)
print(balance_check)
print("\n각 연령대 내에서 A/B가 균형있게 배정됨")

---

## 1.4 표본 크기 계산 (Sample Size Calculation)

### 왜 표본 크기가 중요한가?

- **너무 작으면**: 실제 효과가 있어도 통계적으로 유의하게 나오지 않음 (Type II 오류)
- **너무 크면**: 비용과 시간 낭비, 사소한 차이도 유의하게 나옴

### 표본 크기 결정 요소

| 요소 | 설명 | 일반적 값 |
|------|------|----------|
| 유의수준 (α) | Type I 오류 허용 확률 | 0.05 (5%) |
| 검정력 (1-β) | 실제 효과를 감지할 확률 | 0.80 (80%) |
| 기준 전환율 | 현재 전환율 | 예: 3% |
| 최소 탐지 효과 (MDE) | 감지하려는 최소 개선폭 | 예: 0.5%p |

### statsmodels로 표본 크기 계산

In [ ]:
# 표본 크기 계산 함수
def calculate_sample_size(baseline_rate, mde, alpha=0.05, power=0.80):
    """
    A/B 테스트에 필요한 표본 크기 계산
    
    Parameters:
    - baseline_rate: 기존 전환율 (예: 0.03 = 3%)
    - mde: 최소 탐지 효과 (예: 0.005 = 0.5%p)
    - alpha: 유의수준 (기본값 0.05)
    - power: 검정력 (기본값 0.80)
    
    Returns:
    - 각 그룹에 필요한 표본 크기
    """
    # 효과 크기 계산 (Cohen's h)
    effect_size = proportion_effectsize(baseline_rate, baseline_rate + mde)
    
    # 표본 크기 계산
    analysis = NormalIndPower()
    sample_size = analysis.solve_power(
        effect_size=effect_size,
        alpha=alpha,
        power=power,
        ratio=1.0,  # 두 그룹 크기 동일
        alternative='two-sided'
    )
    
    return int(np.ceil(sample_size))

# 예시: 전환율 3% → 3.5% 개선 감지에 필요한 표본 크기
baseline = 0.03  # 3%
mde = 0.005  # 0.5%p 개선

required_n = calculate_sample_size(baseline, mde)

print("표본 크기 계산 결과")
print("=" * 50)
print(f"기준 전환율: {baseline:.1%}")
print(f"목표 전환율: {baseline + mde:.1%}")
print(f"최소 탐지 효과 (MDE): {mde:.1%}p")
print(f"유의수준 (α): 0.05")
print(f"검정력 (1-β): 0.80")
print(f"\n필요 표본 크기: 각 그룹 {required_n:,}명")
print(f"총 필요 표본: {required_n * 2:,}명")

### 💡 실무 예시: 다양한 시나리오별 표본 크기

In [ ]:
# 다양한 MDE에 따른 표본 크기 비교
baseline = 0.03
mde_values = [0.003, 0.005, 0.01, 0.015, 0.02]  # 0.3%p ~ 2%p

results = []
for mde in mde_values:
    n = calculate_sample_size(baseline, mde)
    relative_lift = mde / baseline * 100
    results.append({
        'MDE': f"{mde:.1%}p",
        '상대적 개선율': f"{relative_lift:.0f}%",
        '목표 전환율': f"{baseline + mde:.1%}",
        '각 그룹 필요 표본': f"{n:,}명",
        '총 필요 표본': f"{n*2:,}명"
    })

sample_size_df = pd.DataFrame(results)
print("MDE별 필요 표본 크기 (기준 전환율 3%)")
print("=" * 70)
print(sample_size_df.to_string(index=False))
print("\n작은 효과를 감지하려면 더 많은 표본이 필요합니다!")

---

## 1.5 실험 기간 설정

### 실험 기간 결정 요소

1. **일일 트래픽**: 필요 표본 수 / 일일 방문자 수
2. **주기성**: 최소 1주일 (주중/주말 패턴 포함)
3. **외부 이벤트**: 세일, 공휴일 등 비정상 기간 피하기
4. **비즈니스 주기**: 월말, 분기말 등 특수 기간 고려

In [ ]:
# 실험 기간 계산기
def calculate_experiment_duration(required_sample_per_group, daily_traffic, traffic_split=0.5):
    """
    실험 기간 계산
    
    Parameters:
    - required_sample_per_group: 각 그룹에 필요한 표본 수
    - daily_traffic: 일일 방문자 수
    - traffic_split: 실험에 할당할 트래픽 비율 (기본 50%)
    
    Returns:
    - 필요 실험 일수
    """
    daily_per_group = daily_traffic * traffic_split * 0.5  # 50:50 분배
    days_needed = np.ceil(required_sample_per_group / daily_per_group)
    
    # 최소 7일 (주기성 고려)
    return max(7, int(days_needed))

# 예시: 일일 방문자 10,000명, 필요 표본 5,000명/그룹
daily_visitors = 10000
required_n = 5000

experiment_days = calculate_experiment_duration(required_n, daily_visitors)

print("실험 기간 계산")
print("=" * 50)
print(f"일일 방문자: {daily_visitors:,}명")
print(f"실험 트래픽 비율: 100% (전체 트래픽 사용)")
print(f"일일 각 그룹 배정: {daily_visitors * 0.5:,.0f}명")
print(f"필요 표본/그룹: {required_n:,}명")
print(f"\n최소 실험 기간: {experiment_days}일")
print(f"권장 실험 기간: {experiment_days + 7}일 (여유 포함)")

### 💡 실무 예시: 실험 설계서 작성

In [ ]:
# 완전한 A/B 테스트 설계서
experiment_plan = {
    '항목': [
        '실험명',
        '목적',
        '가설',
        '대조군 (A)',
        '실험군 (B)',
        '주요 지표 (Primary Metric)',
        '보조 지표 (Secondary Metrics)',
        '표본 크기',
        '실험 기간',
        '성공 기준'
    ],
    '내용': [
        '결제 버튼 색상 최적화 테스트',
        '결제 전환율 향상',
        '초록색 버튼이 파란색 버튼보다 전환율이 높다',
        '기존 파란색 버튼 (#0066CC)',
        '신규 초록색 버튼 (#00CC66)',
        '결제 전환율 (구매 완료 / 결제 페이지 방문)',
        '클릭률, 평균 주문금액, 이탈률',
        '각 그룹 5,000명 (총 10,000명)',
        '14일 (2주)',
        '전환율 5% 이상 개선, p-value < 0.05'
    ]
}

plan_df = pd.DataFrame(experiment_plan)
print("A/B 테스트 설계서")
print("=" * 70)
for _, row in plan_df.iterrows():
    print(f"{row['항목']:20} | {row['내용']}")

---

# Part 2: 평가 지표와 통계적 유의성

---

## 2.1 핵심 평가 지표

### 전환율 (Conversion Rate)

**전환율** = (전환 수 / 방문자 수) × 100

- **이커머스**: 구매 완료 / 사이트 방문
- **SaaS**: 유료 전환 / 무료 가입
- **앱**: 회원가입 / 앱 다운로드

In [4]:
# 전환율 계산 예시
ab_data = {
    'group': ['Control', 'Treatment'],
    'visitors': [10000, 10000],
    'conversions': [300, 380]
}

df = pd.DataFrame(ab_data)
df['conversion_rate'] = df['conversions'] / df['visitors'] * 100

print("전환율 계산")
print("=" * 50)
for _, row in df.iterrows():
    print(f"{row['group']}: {row['conversions']:,}/{row['visitors']:,} = {row['conversion_rate']:.2f}%")

# 절대적 차이 vs 상대적 차이
absolute_diff = df.loc[1, 'conversion_rate'] - df.loc[0, 'conversion_rate']
relative_diff = (df.loc[1, 'conversion_rate'] - df.loc[0, 'conversion_rate']) / df.loc[0, 'conversion_rate'] * 100

print(f"\n절대적 차이: {absolute_diff:.2f}%p (3.00% → 3.80%)")
print(f"상대적 차이 (Lift): {relative_diff:.1f}% 개선")

전환율 계산
Control: 300/10,000 = 3.00%
Treatment: 380/10,000 = 3.80%

절대적 차이: 0.80%p (3.00% → 3.80%)
상대적 차이 (Lift): 26.7% 개선


### 클릭률 (CTR: Click-Through Rate)

**CTR** = (클릭 수 / 노출 수) × 100

- **광고**: 광고 클릭 / 광고 노출
- **이메일**: 링크 클릭 / 이메일 오픈
- **검색**: 검색 결과 클릭 / 검색 결과 노출

In [5]:
# 이메일 마케팅 CTR 비교
email_test = pd.DataFrame({
    '그룹': ['A (기존 제목)', 'B (신규 제목)'],
    '발송': [50000, 50000],
    '오픈': [10000, 12500],
    '클릭': [500, 750]
})

email_test['오픈율'] = email_test['오픈'] / email_test['발송'] * 100
email_test['CTR(오픈 대비)'] = email_test['클릭'] / email_test['오픈'] * 100
email_test['CTR(발송 대비)'] = email_test['클릭'] / email_test['발송'] * 100

print("이메일 A/B 테스트 결과")
print("=" * 60)
print(email_test[['그룹', '발송', '오픈', '클릭', '오픈율', 'CTR(오픈 대비)']].to_string(index=False))

이메일 A/B 테스트 결과
       그룹    발송    오픈  클릭  오픈율  CTR(오픈 대비)
A (기존 제목) 50000 10000 500 20.0         5.0
B (신규 제목) 50000 12500 750 25.0         6.0


### 평균 주문 금액 (AOV: Average Order Value)

**AOV** = 총 매출 / 주문 수

전환율이 올랐어도 AOV가 떨어지면 전체 매출은 감소할 수 있음!

In [6]:
# AOV 분석
aov_data = pd.DataFrame({
    '그룹': ['Control', 'Treatment'],
    '주문 수': [300, 380],
    '총 매출(만원)': [15000, 17100]
})

aov_data['AOV(만원)'] = aov_data['총 매출(만원)'] / aov_data['주문 수']

print("AOV 분석")
print("=" * 50)
print(aov_data.to_string(index=False))

# 전환율은 올랐지만 AOV는?
conv_lift = (380 - 300) / 300 * 100
aov_change = (17100/380 - 15000/300) / (15000/300) * 100

print(f"\n전환 수: {conv_lift:.1f}% 증가")
print(f"AOV: {aov_change:.1f}% 변화")
print(f"총 매출: {(17100-15000)/15000*100:.1f}% 증가")

AOV 분석
       그룹  주문 수  총 매출(만원)  AOV(만원)
  Control   300     15000     50.0
Treatment   380     17100     45.0

전환 수: 26.7% 증가
AOV: -10.0% 변화
총 매출: 14.0% 증가


### 💡 실무 예시: 종합 지표 대시보드

In [7]:
# A/B 테스트 종합 대시보드
dashboard_data = pd.DataFrame({
    '지표': ['방문자', '전환 수', '전환율', '총 매출(만원)', 'AOV(만원)', 'ARPU(만원)'],
    'Control': [10000, 300, '3.00%', 15000, 50.0, 1.50],
    'Treatment': [10000, 380, '3.80%', 17100, 45.0, 1.71],
    '변화': ['-', '+80 (+26.7%)', '+0.80%p (+26.7%)', '+2,100 (+14.0%)', '-5.0 (-10.0%)', '+0.21 (+14.0%)']
})

print("A/B 테스트 종합 대시보드")
print("=" * 70)
print(dashboard_data.to_string(index=False))
print("\n* ARPU (Average Revenue Per User) = 총 매출 / 방문자 수")

A/B 테스트 종합 대시보드
      지표 Control Treatment               변화
     방문자   10000     10000                -
    전환 수     300       380     +80 (+26.7%)
     전환율   3.00%     3.80% +0.80%p (+26.7%)
총 매출(만원)   15000     17100  +2,100 (+14.0%)
 AOV(만원)    50.0      45.0    -5.0 (-10.0%)
ARPU(만원)     1.5      1.71   +0.21 (+14.0%)

* ARPU (Average Revenue Per User) = 총 매출 / 방문자 수


---

## 2.2 통계적 유의성 검정

### 비율 차이 검정 (Z-test for Proportions)

전환율과 같은 **비율(proportion)** 비교에는 Z-test를 사용합니다.

In [8]:
# 두 비율 검정 => A,B 테스트에 많이 씀! 
# statsmodels로 비율 검정
# Control: 10000명 중 300명 전환 (3%)
# Treatment: 10000명 중 380명 전환 (3.8%)

# 성공 수
successes = np.array([300, 380])
# 시도 수
trials = np.array([10000, 10000])

# 비율 z-검정
z_stat, p_value = proportions_ztest(successes, trials, alternative='two-sided')

print("비율 차이 Z-검정 결과")
print("=" * 50)
print(f"Control 전환율: {successes[0]/trials[0]:.2%}")
print(f"Treatment 전환율: {successes[1]/trials[1]:.2%}")
print(f"\nZ-statistic: {z_stat:.4f}")
print(f"p-value: {p_value:.6f}")

alpha = 0.05
if p_value < alpha:
    print(f"\np-value ({p_value:.4f}) < alpha ({alpha})")
    print("결론: Treatment 그룹의 전환율이 통계적으로 유의하게 높습니다!")
else:
    print(f"\np-value ({p_value:.4f}) >= alpha ({alpha})")
    print("결론: 두 그룹 간 통계적으로 유의한 차이가 없습니다.")

비율 차이 Z-검정 결과
Control 전환율: 3.00%
Treatment 전환율: 3.80%

Z-statistic: -3.1214
p-value: 0.001800

p-value (0.0018) < alpha (0.05)
결론: Treatment 그룹의 전환율이 통계적으로 유의하게 높습니다!


### 신뢰구간 (Confidence Interval) 계산

In [9]:
# 전환율 차이의 신뢰구간
from statsmodels.stats.proportion import confint_proportions_2indep

# 전환율 차이의 95% 신뢰구간
p1 = successes[0] / trials[0]  # Control
p2 = successes[1] / trials[1]  # Treatment
n1, n2 = trials[0], trials[1]

# 신뢰구간 계산 (수동)
diff = p2 - p1
se = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
ci_low = diff - 1.96 * se
ci_high = diff + 1.96 * se

print("전환율 차이 신뢰구간")
print("=" * 50)
print(f"전환율 차이: {diff:.2%}p")
print(f"95% 신뢰구간: [{ci_low:.2%}p, {ci_high:.2%}p]")

if ci_low > 0:
    print("\n신뢰구간이 0을 포함하지 않음")
    print("Treatment가 Control보다 유의하게 높음!")
elif ci_high < 0:
    print("\n신뢰구간이 0을 포함하지 않음")
    print("Treatment가 Control보다 유의하게 낮음!")
else:
    print("\n신뢰구간이 0을 포함함")
    print("통계적으로 유의한 차이 없음")

전환율 차이 신뢰구간
전환율 차이: 0.80%p
95% 신뢰구간: [0.30%p, 1.30%p]

신뢰구간이 0을 포함하지 않음
Treatment가 Control보다 유의하게 높음!


### 💡 실무 예시: Plotly로 A/B 테스트 결과 시각화

In [10]:
# A/B 테스트 결과 시각화
groups = ['Control', 'Treatment']
conversion_rates = [3.0, 3.8]
ci_errors = [0.34, 0.38]  # 95% CI 반폭

fig = go.Figure()

# 막대 그래프
fig.add_trace(go.Bar(
    x=groups,
    y=conversion_rates,
    error_y=dict(
        type='data',
        array=ci_errors,
        visible=True
    ),
    marker_color=['#3498db', '#2ecc71'],
    text=[f'{r:.1f}%' for r in conversion_rates],
    textposition='outside'
))

fig.update_layout(
    title='A/B 테스트 결과: 전환율 비교 (95% 신뢰구간)',
    xaxis_title='그룹',
    yaxis_title='전환율 (%)',
    yaxis=dict(range=[0, 5]),
    showlegend=False,
    width=600,
    height=450
)

# 통계적 유의성 표시
fig.add_annotation(
    x=0.5, y=4.5,
    text=f"p-value = {p_value:.4f} (유의함)",
    showarrow=False,
    font=dict(size=14, color='green')
)

fig.show()

---

## 2.3 검정력(Power)과 효과 크기(Effect Size)

### 검정력 (Statistical Power)

**검정력 (1-β)**: 실제 효과가 있을 때 이를 탐지할 확률

- **1-β = 0.80**: 실제 효과가 있으면 80% 확률로 탐지
- 검정력이 낮으면 → 효과가 있어도 "효과 없음"으로 잘못 결론 (Type II 오류)

### 효과 크기 (Effect Size)

비율 비교에서는 **Cohen's h**를 사용:

$$h = 2 \times (\arcsin(\sqrt{p_2}) - \arcsin(\sqrt{p_1}))$$

| Cohen's h | 해석 |
|-----------|------|
| 0.2 | 작은 효과 |
| 0.5 | 중간 효과 |
| 0.8 | 큰 효과 |

In [12]:
# 효과 크기 계산
p1 = 0.03  # Control 3%
p2 = 0.038  # Treatment 3.8%

# Cohen's h
effect_size = proportion_effectsize(p1, p2)

print("효과 크기 분석")
print("=" * 50)
print(f"Control 전환율: {p1:.1%}")
print(f"Treatment 전환율: {p2:.1%}")
print(f"Cohen's h: {abs(effect_size):.4f}")

if abs(effect_size) < 0.2:
    print("효과 크기: 매우 작음")
elif abs(effect_size) < 0.5:
    print("효과 크기: 작음")
elif abs(effect_size) < 0.8:
    print("효과 크기: 중간")
else:
    print("효과 크기: 큼")

효과 크기 분석
Control 전환율: 3.0%
Treatment 전환율: 3.8%
Cohen's h: 0.0442
효과 크기: 매우 작음


In [13]:
# 검정력 분석: 현재 표본 크기로 효과를 탐지할 수 있는가?
from statsmodels.stats.power import NormalIndPower

analysis = NormalIndPower()

# 주어진 조건에서 검정력 계산
power = analysis.solve_power(
    effect_size=abs(effect_size),
    nobs1=10000,  # 각 그룹 표본 크기
    alpha=0.05,
    ratio=1.0,
    alternative='two-sided'
)

print(f"\n현재 표본 크기(n=10,000/그룹)에서의 검정력: {power:.1%}")

if power >= 0.80:
    print("충분한 검정력 확보 (80% 이상)")
else:
    print(f"검정력 부족! 권장 표본 크기 계산 필요")


현재 표본 크기(n=10,000/그룹)에서의 검정력: 87.8%
충분한 검정력 확보 (80% 이상)


### 💡 실무 예시: 검정력 곡선 시각화

In [14]:
# 표본 크기에 따른 검정력 변화
sample_sizes = np.arange(1000, 20001, 1000)
powers = []

for n in sample_sizes:
    power = analysis.solve_power(
        effect_size=abs(effect_size),
        nobs1=n,
        alpha=0.05,
        ratio=1.0,
        alternative='two-sided'
    )
    powers.append(power)

# Plotly로 검정력 곡선 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sample_sizes,
    y=powers,
    mode='lines+markers',
    name='검정력',
    line=dict(color='#3498db', width=2),
    marker=dict(size=6)
))

# 80% 검정력 기준선
fig.add_hline(y=0.80, line_dash='dash', line_color='red',
              annotation_text='권장 검정력 80%')

fig.update_layout(
    title='표본 크기에 따른 검정력 변화 (효과 크기 고정)',
    xaxis_title='각 그룹 표본 크기',
    yaxis_title='검정력 (Statistical Power)',
    yaxis=dict(tickformat='.0%', range=[0, 1]),
    width=700,
    height=450
)

fig.show()

---

## 2.4 A/B 테스트 실무 함정 (Pitfalls)

### 1. 조기 종료 (Peeking Problem)

실험 중간에 p-value를 확인하고 조기 종료하면 **False Positive** 증가!

In [ ]:
# Peeking 시뮬레이션: 실제 효과가 없는데도 유의하게 나올 수 있음
np.random.seed(42)

# 실제로는 두 그룹 모두 전환율 3% (차이 없음)
true_rate = 0.03
n_simulations = 1000
false_positives = 0

for _ in range(n_simulations):
    # 매일 100명씩 14일간 데이터 수집
    control_conv = 0
    treatment_conv = 0
    
    for day in range(14):
        control_conv += np.random.binomial(100, true_rate)
        treatment_conv += np.random.binomial(100, true_rate)
        
        # 매일 peeking (p-value 확인)
        n = (day + 1) * 100
        if n >= 200:  # 최소 표본
            z, p = proportions_ztest([control_conv, treatment_conv], [n, n])
            if p < 0.05:
                false_positives += 1
                break  # 조기 종료

print("Peeking Problem 시뮬레이션 결과")
print("=" * 50)
print(f"시뮬레이션 횟수: {n_simulations}")
print(f"실제 효과: 없음 (두 그룹 모두 3%)")
print(f"매일 peeking 시 False Positive: {false_positives/n_simulations:.1%}")
print(f"\n원래 Type I 오류율: 5%")
print(f"Peeking으로 인한 실제 오류율: {false_positives/n_simulations:.1%}")
print("\n매일 peeking하면 오류율이 5%보다 훨씬 높아짐!")

### 2. 다중 비교 문제 (Multiple Comparisons)

여러 지표를 동시에 테스트하면 하나 이상이 우연히 유의하게 나올 확률 증가

FWER ??

해결 방법은 유의 수준을 낮춰버리는 것 ! 

In [ ]:
# 다중 비교 문제 시각화
n_metrics = range(1, 21)
alpha = 0.05

# 적어도 하나의 False Positive가 나올 확률
# P(at least one FP) = 1 - P(no FP) = 1 - (1-alpha)^n
familywise_error = [1 - (1 - alpha) ** n for n in n_metrics]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(n_metrics),
    y=familywise_error,
    mode='lines+markers',
    name='오류 확률',
    line=dict(color='#e74c3c', width=2)
))

fig.add_hline(y=0.05, line_dash='dash', line_color='green',
              annotation_text='원래 유의수준 5%')

fig.update_layout(
    title='다중 비교 문제: 테스트 지표 수에 따른 오류율 증가',
    xaxis_title='테스트하는 지표 수',
    yaxis_title='적어도 하나의 False Positive 확률',
    yaxis=dict(tickformat='.0%'),
    width=700,
    height=450
)

fig.show()

print(f"\n예시: 10개 지표를 테스트하면")
print(f"적어도 하나가 우연히 유의하게 나올 확률: {1-(1-0.05)**10:.1%}")

### 3. Simpson's Paradox (심슨의 역설)

전체 데이터에서는 B가 좋아 보이지만, 세그먼트별로 보면 A가 더 좋을 수 있음!

In [ ]:
# Simpson's Paradox 예시
simpsons_data = pd.DataFrame({
    '세그먼트': ['모바일', '모바일', '데스크톱', '데스크톱', '전체', '전체'],
    '그룹': ['A', 'B', 'A', 'B', 'A', 'B'],
    '방문자': [8000, 2000, 2000, 8000, 10000, 10000],
    '전환': [400, 120, 80, 400, 480, 520],
})

simpsons_data['전환율'] = simpsons_data['전환'] / simpsons_data['방문자'] * 100

print("Simpson's Paradox 예시")
print("=" * 60)
print(simpsons_data.to_string(index=False))

print("\n분석:")
print("- 모바일: A(5.0%) > B(6.0%) -- B가 더 좋음")
print("- 데스크톱: A(4.0%) < B(5.0%) -- B가 더 좋음")
print("- 전체: A(4.8%) < B(5.2%) -- B가 더 좋음")
print("\n이 경우는 B가 일관되게 좋음 (역설 아님)")
print("\n역설이 발생하는 경우:")
print("세그먼트별로는 A가 좋은데, 전체로 보면 B가 좋을 수 있음!")
print("항상 세그먼트별로 분석하세요!")

### 4. 신규성 효과 (Novelty Effect)

새로운 것이라서 일시적으로 클릭이 증가하는 현상

In [ ]:
# Novelty Effect 시뮬레이션
days = list(range(1, 22))
control_daily = [3.0] * 21  # Control: 일정한 3%

# Treatment: 초기 높다가 점점 감소
treatment_daily = [5.0 - 0.1 * d for d in range(21)]
treatment_daily = [max(3.0, t) for t in treatment_daily]  # 최소 3%

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=days, y=control_daily,
    mode='lines+markers',
    name='Control',
    line=dict(color='#3498db')
))

fig.add_trace(go.Scatter(
    x=days, y=treatment_daily,
    mode='lines+markers',
    name='Treatment (Novelty Effect)',
    line=dict(color='#e74c3c')
))

fig.update_layout(
    title='Novelty Effect: 신규성으로 인한 일시적 전환율 상승',
    xaxis_title='실험 일차',
    yaxis_title='전환율 (%)',
    yaxis=dict(range=[0, 6]),
    width=700,
    height=450
)

fig.show()

print("\nNovelty Effect 대응:")
print("1. 충분히 긴 실험 기간 (2-4주)")
print("2. 시간에 따른 효과 변화 모니터링")
print("3. 신규 사용자와 기존 사용자 분리 분석")

### 💡 실무 함정 체크리스트

In [ ]:
# A/B 테스트 함정 체크리스트
pitfalls_checklist = pd.DataFrame({
    '함정': [
        'Peeking (조기 종료)',
        '다중 비교',
        'Simpson\'s Paradox',
        'Novelty Effect',
        '표본 크기 부족',
        '무작위 배정 실패'
    ],
    '증상': [
        'False Positive 증가',
        '우연히 유의한 결과',
        '세그먼트별 결과 상이',
        '초기 효과 과대평가',
        '결과 불안정',
        '그룹 간 특성 불균형'
    ],
    '대응': [
        '실험 기간 사전 결정 후 준수',
        'Bonferroni 보정 적용',
        '세그먼트별 분석 병행',
        '2주 이상 실험, 기존 사용자 분리',
        '사전 Power Analysis',
        'AA 테스트로 검증'
    ]
})

print("A/B 테스트 실무 함정 체크리스트")
print("=" * 70)
print(pitfalls_checklist.to_string(index=False))

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 전환율 계산 ⭐

**문제**: 다음 A/B 테스트 데이터에서 각 그룹의 전환율을 계산하세요.

```python
control_visitors = 5000
control_conversions = 150
treatment_visitors = 5000
treatment_conversions = 200
```

**출력**: 각 그룹 전환율(%), 절대적 차이(%p), 상대적 개선율(%)

In [ ]:
control_visitors = 5000
control_conversions = 150
treatment_visitors = 5000
treatment_conversions = 200

# 여기에 코드를 작성하세요


### Q2. 무작위 배정 검증 ⭐⭐

**문제**: 1000명의 사용자를 A/B 그룹에 배정하고, 각 그룹의 비율이 약 50:50인지 확인하세요.

**요구사항**: `np.random.choice` 또는 `np.random.binomial` 사용

In [ ]:
np.random.seed(42)
n_users = 1000

# 여기에 코드를 작성하세요


### Q3. 실험 기간 계산 ⭐⭐

**문제**: 일일 방문자 5,000명인 사이트에서 각 그룹에 3,000명이 필요할 때, 최소 실험 기간을 계산하세요.

**조건**: 
- 전체 트래픽의 100% 실험에 사용
- 50:50 분배
- 최소 7일 (주기성 고려)

In [ ]:
daily_traffic = 5000
required_per_group = 3000

# 여기에 코드를 작성하세요


### Q4. 비율 검정 (Z-test) ⭐⭐⭐

**문제**: 다음 A/B 테스트 결과가 통계적으로 유의한지 `proportions_ztest`로 검정하세요.

```python
control = {'visitors': 8000, 'conversions': 240}  # 3%
treatment = {'visitors': 8000, 'conversions': 320}  # 4%
```

**출력**: Z-statistic, p-value, 결론

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

control = {'visitors': 8000, 'conversions': 240}
treatment = {'visitors': 8000, 'conversions': 320}

# 여기에 코드를 작성하세요


### Q5. 신뢰구간 계산 ⭐⭐⭐

**문제**: Q4의 전환율 차이에 대한 95% 신뢰구간을 계산하세요.

**힌트**: 
- 차이 = p2 - p1
- SE = sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
- 95% CI = 차이 +/- 1.96 * SE

In [ ]:
# Q4 데이터 사용
p1 = 240 / 8000  # Control
p2 = 320 / 8000  # Treatment
n1 = n2 = 8000

# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 표본 크기 계산 ⭐⭐⭐

**문제**: 현재 전환율 2%에서 2.5%로 개선을 감지하려면 각 그룹에 몇 명이 필요한지 계산하세요.

**조건**: alpha=0.05, power=0.80

In [ ]:
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import NormalIndPower

baseline_rate = 0.02
target_rate = 0.025

# 여기에 코드를 작성하세요


### Q7. 검정력 계산 ⭐⭐⭐

**문제**: 각 그룹 5,000명으로 3% → 3.5% 개선을 감지할 수 있는 검정력을 계산하세요.

In [ ]:
n_per_group = 5000
p1 = 0.03
p2 = 0.035

# 여기에 코드를 작성하세요


### Q8. A/B 테스트 결과 시각화 ⭐⭐⭐⭐

**문제**: 다음 A/B 테스트 결과를 Plotly 막대 그래프로 시각화하세요. 신뢰구간(error bar) 포함.

```python
results = {
    '그룹': ['Control', 'Treatment'],
    '전환율': [2.5, 3.2],
    'CI_error': [0.3, 0.35]  # 95% CI 반폭
}
```

In [ ]:
results = {
    '그룹': ['Control', 'Treatment'],
    '전환율': [2.5, 3.2],
    'CI_error': [0.3, 0.35]
}

# 여기에 코드를 작성하세요


### Q9. 세그먼트별 분석 ⭐⭐⭐⭐

**문제**: 다음 데이터를 세그먼트별(모바일/데스크톱)과 전체로 분석하여 Simpson's Paradox가 있는지 확인하세요.

```python
segment_data = {
    'segment': ['Mobile', 'Mobile', 'Desktop', 'Desktop'],
    'group': ['A', 'B', 'A', 'B'],
    'visitors': [6000, 4000, 4000, 6000],
    'conversions': [180, 160, 200, 240]
}
```

**출력**: 세그먼트별 전환율, 전체 전환율, Simpson's Paradox 여부

In [ ]:
segment_data = {
    'segment': ['Mobile', 'Mobile', 'Desktop', 'Desktop'],
    'group': ['A', 'B', 'A', 'B'],
    'visitors': [6000, 4000, 4000, 6000],
    'conversions': [180, 160, 200, 240]
}

# 여기에 코드를 작성하세요


### Q10. 종합 A/B 테스트 분석 ⭐⭐⭐⭐⭐

**문제**: 다음 마케팅 캠페인 A/B 테스트 데이터를 완전히 분석하세요.

```python
campaign_test = {
    'Control': {'visitors': 12000, 'clicks': 600, 'conversions': 120, 'revenue': 24000000},
    'Treatment': {'visitors': 12000, 'clicks': 840, 'conversions': 168, 'revenue': 30240000}
}
```

**분석 항목**:
1. CTR, 전환율, AOV, ARPU 계산
2. 전환율 차이의 통계적 유의성 검정 (p-value)
3. 95% 신뢰구간
4. 상대적 개선율 (Lift)
5. 비즈니스 인사이트 및 권장사항

In [ ]:
campaign_test = {
    'Control': {'visitors': 12000, 'clicks': 600, 'conversions': 120, 'revenue': 24000000},
    'Treatment': {'visitors': 12000, 'clicks': 840, 'conversions': 168, 'revenue': 30240000}
}

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 실험 설계 핵심 요약

| 개념 | 설명 | 핵심 포인트 |
|------|------|-------------|
| A/B 테스트 | 두 버전을 비교하는 통제 실험 | 단일 변수만 변경 |
| 대조군 (Control) | 기존 버전, 변화 없음 | Baseline으로 비교 기준 |
| 실험군 (Treatment) | 새로운 버전, 변화 적용 | 효과 측정 대상 |
| 무작위 배정 | 사용자를 랜덤하게 그룹 배정 | 선택 편향 방지 |
| 표본 크기 | 통계적 유의성 확보에 필요한 수 | Power Analysis로 사전 계산 |
| 실험 기간 | 최소 1-2주 권장 | 주기성, 외부 요인 고려 |

### Part 2: 통계적 유의성 핵심 요약

| 개념 | Python 함수 | 언제 쓰나? |
|------|-------------|------------|
| 전환율 | 직접 계산 | 모든 A/B 테스트의 기본 지표 |
| 비율 Z-검정 | `proportions_ztest()` | 전환율 차이 유의성 검정 |
| 효과 크기 | `proportion_effectsize()` | Cohen's h 계산 |
| 검정력 분석 | `NormalIndPower()` | 표본 크기, 검정력 계산 |
| 신뢰구간 | 수동 계산 | 효과 크기의 범위 추정 |

### 💡 실무 팁

1. **사전 계획 필수**: 실험 전에 표본 크기, 기간, 성공 기준 정의
2. **Peeking 금지**: 실험 중간에 결과 보고 조기 종료하지 않기
3. **다중 비교 보정**: 여러 지표 테스트 시 Bonferroni 보정 적용
4. **세그먼트 분석**: 전체뿐 아니라 세그먼트별 결과 확인 (Simpson's Paradox 방지)
5. **Novelty Effect 주의**: 2주 이상 실험, 기존 사용자 별도 분석
6. **실무적 유의성**: p-value뿐 아니라 효과 크기와 비즈니스 임팩트 고려
7. **AA 테스트**: 실험 시스템 검증을 위해 A-A 테스트 먼저 수행

### 📈 A/B 테스트 의사결정 플로우

```
1. 가설 설정
    ↓
2. 표본 크기 계산 (Power Analysis)
    ↓
3. 실험 설계 (기간, 트래픽 분배)
    ↓
4. 무작위 배정 및 실험 실행
    ↓
5. 데이터 수집 (Peeking 금지!)
    ↓
6. 통계 검정 (Z-test, 신뢰구간)
    ↓
7. 세그먼트별 분석
    ↓
8. 의사결정 및 적용
```